<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#KcBERT 학습용 데이터셋을 제작하는 코드(로컬)
#제작할 데이터셋은 pretraining용 데이터셋(soynlp 학습용 데이터), finetuning용 데이터셋(next sentence prediction, text classification)

In [ ]:
!pip install kss
!pip install datasets
!pip install soynlp
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from soynlp.normalizer import *
from kss import Kss
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction, TrainingArguments, BertForSequenceClassification
from datasets import Dataset, load_dataset, ClassLabel
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
#전처리된 파일 로드(BP_spacing 건너뛰기)
talk_save_path = '/content/drive/My Drive/talk_preprocess_result_short.xlsx'
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
all_contents_path = '/content/drive/My Drive/community_all_contents_preprocessed.xlsx'
all_comments_path = '/content/drive/My Drive/community_all_comments_preprocessed.xlsx'

df_talk = pd.read_excel(talk_save_path)
df_qna = pd.read_excel(qna_path)
df_all_contents = pd.read_excel(all_contents_path)
df_all_comments = pd.read_excel(all_comments_path)

In [ ]:
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
df_qna = pd.read_excel(qna_path)

In [ ]:
#직접 제작한 카카오톡 질문답변 데이터셋 로드
talk_finetune_path = '/content/drive/My Drive/talk_finetune_dataset.xlsx'

df_talk_finetune = pd.read_excel(talk_finetune_path, index_col = 0)

In [ ]:
#직접 제작한 카카오톡 질문 데이터셋 로드
talk_question_finetune_path = '/content/drive/My Drive/talk_finetune_question_dataset.xlsx'

df_talk_finetune_question = pd.read_excel(talk_question_finetune_path, index_col = 0)

#Soyspacing 학습시킨 후에 띄어쓰기 교정

In [ ]:
#일단 보류

#Pretrain 데이터셋, 제공받은 모든 데이터 사용


In [ ]:
split_sentences = Kss('split_sentences')

In [ ]:
#kss로 리스트 안에 있는 여러 문장으로 이루어진 글을 문장 단위로 분리하는 함수
def separate_to_sentences_and_deduplicate(DataFrame, columns):
    sentences_list = []
    for column in columns: #데이터프레임의 각 열에 대해서 처리
        text_list = DataFrame[column].to_list()
        sentences_list.append(split_sentences(text_list))
    return list(set(sum([lists for lists in sentences_list[0]], [])))

In [ ]:
#데이터프레임의 열들을 리스트로 변환
talk_list = separate_to_sentences_and_deduplicate(df_talk, ['text'])


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
#데이터프레임의 NaN 대신 '' 채워넣기
df_qna.fillna('', inplace = True)
df_all_contents.fillna('', inplace = True)
df_all_comments.fillna('', inplace = True)

In [ ]:
qna_list = separate_to_sentences_and_deduplicate(df_qna, ['내용', '답변', '댓글'])

In [ ]:
all_contents_list = separate_to_sentences_and_deduplicate(df_all_contents, ['내용', '답변', '댓글'])

In [ ]:
all_comments_list = separate_to_sentences_and_deduplicate(df_all_comments, ['내용', '대댓글', '댓글'])
print(all_comments_list)

['너무 고맙습니다.', '레이아웃을 전부 fixed로 하시는 이유가 있을까요??', '축하드려요!', '사실 지금도 하나가 걸리면 해결하기 위해 시간을 쓰곤 하고요.', '대표님 두분이 댓글달아주셔서 영광입니다!♡', '보니까 data type이 text로 되어 있는데 json으로 바꿔서 해보시겠어요?', '저희 팀은 믹스패널에 이벤트를 쏴서 믹스패널 내에서 보고 있습니다.', '언제나 환영입니다.', 'get data from url로 하니 계속 데이터 타입이 안맞는다고 해서요, 혹시 조금만 더 구체적으로 알려주실 수 있을까요?', '컨설팅 또 해줘', '혹은 데이터의 privacy rule 때문일 수도 있습니다.', '바로 시도해보겠습니다~!', '앞으로도 꾸준히 공유드리겠습니다', '한번에 해결하려하기보다는 내부에 있는 요소의 크기를 가늠하셔서 \n전부 더한 값에 -값을 주면서 프리뷰로 체크해보시면서 맞춰보시면 금방 하실 겁니다.', '슬랙이  내부 알림용 채널로 정말 좋은 것 같습니다.', 'ㅠ..', ':)\n네 맞습니다', '와우...', '이렇게 설정하시면 모든 페이지들에서 사용이 가능하지만, 필요없는 페이지에서도 SDK를 호출하기 때문에 추천드리고 싶은 방식은 아닙니다.', '준호님 하고 싶은 질문이 너무 많은데 줄여서 이렇게 정리해봤습니다!', '저거 보셧을때는 문제가없나요?', '물론 양날의 검입니다.', '(놀면서 틈틈히한거지만) 다만 저희가 좋아하는 분야다보니 재밌게했었습니다. ㅎㅎ', '굳이 고집하실 필요 없어요.', '혹시 gpt 연동하실 때는 어떤 레퍼런스 참고하셨는지 여쭤봐도 될까요?', '해치하이커님한테 여쭤보시는 게 빠를 것 같아요.', '저도 가끔씩 기본적인 것에서 막히면 라이언님 글 보고 다시 나오곤 합니다ㅎ!', '어떤 계기로 버블을 배워보기로 하셨는지 궁금하고, 아울러 개발 백그라운드를 가진 분이 버블을 배워나가시면 어떤지 많이 궁금합니다.', '이런 부분까지도 챗지피티가 해주나요?', '그리고 귀띔인데 요즘은 chat

In [ ]:
#하나의 리스트로 결합
result_list = talk_list + qna_list + all_contents_list + all_comments_list
print(result_list[:10])
print(len(result_list))
df_result = pd.DataFrame(result_list, columns = ['text'])
print(df_result)
print(len(df_result))

['잼나네요', '근데 line space를 1.2 로주면 line height 이 1.2 가 그대로 되네요?', '앗 뭔가 했네요 ㅋㅋㅋㅋㅋㅋㅋ', '긍정(신기하다, 대단하다) VS 부정(그런 거 되겠냐)', 'font-family 이름이랑 업로드하실때 넣는 Font name이랑 매치해야되는 것도 잘 알려주지 않는 함정입니다', '네 ㅎㅎ 편하신 걸로 하시면 될 듯 합니다~', '혹시 아시는분 답변 주시면 정말 감사드리겠습니다..ㅠㅠ', '저기 월을 누르면 해당되는 월만 나오게 하는 거에요', '해당 multiline input을 지우고 새로운 multiline input을 만드니 되네요...', '방에서 열심히 질의응답 해주시는 선생님들도 얼마나 학습하는데 고생하셨을텐데']
29334
                                                    text
0                                                   잼나네요
1      근데 line space를 1.2 로주면 line height 이 1.2 가 그대로...
2                                       앗 뭔가 했네요 ㅋㅋㅋㅋㅋㅋㅋ
3                         긍정(신기하다, 대단하다) VS 부정(그런 거 되겠냐)
4      font-family 이름이랑 업로드하실때 넣는 Font name이랑 매치해야되는 ...
...                                                  ...
29329                               간단한 백엔드 구성이 필요해보입니다.
29330                             아니 이렇게 정성스럽게 정리해주시다니..
29331                                             이상하게..
29332                                   

In [ ]:
#전체 데이터에 대해 soynlp.normalize 적용?
#일단 보류

In [ ]:
#.csv 파일로 저장(.xlsx로 저장 시 illegalcharactererror 발생)
pretrain_data_save_path = '/content/drive/My Drive/KcBERT_pretrain_dataset_short.csv'

df_result.to_csv(pretrain_data_save_path)

#Finetune 데이터셋(질문-답변)


In [ ]:
#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[pd.isna(df_qna['Slug']) == False]
df_question_answer = df_question_answer[pd.isna(df_question_answer['답변']) == False]

print(df_question_answer)
print(len(df_question_answer))

print(df_question_answer.iloc[0])
print(df_question_answer.iloc[1]['답변'])

     Unnamed: 0                                                 내용  \
0             0  안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1             1  저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
2             2  안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3             3  헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...   
4             4  안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
..          ...                                                ...   
573         573  정보 구조나 와이어 프레임을 그릴 때 보통 피그마를 쓰시는 분이 많은데 한글로 된 ...   
574         574                                                      
575         575  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
576         576  안녕하세요! 버블로 외주개발하고 있는 리트머스팀입니다.yongjun@cigro.io...   
577         577                 안녕하세요~아래 링크로 톡 남겨놓으시면 견적 도와드리겠습니다!   

                                                    답변  \
0    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
1                                          

질문 + 답변에다가 답변에 달린 댓글들까지 입력 데이터로 처리할 경우 퀄리티가 더 좋아질 수 있지만, max_length를 초과하지 않는 방향에 따라 답변 데이터만 이용, 나중에 개선방안 찾아보기

community_qna_preprocessed 데이터에서 이미지 파일을 base64 인코딩한 텍스트가 섞여 있는데, 데이터 파일 추출 과정에서 직접 제거하는 게 좋을 것 같음.

In [ ]:
#커뮤니티 파일을 가공해서 finetune 데이터셋을 제작
finetune_dict = {}

#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[df_qna['Slug'] != '']
df_question_answer = df_question_answer[df_question_answer['답변'] != '']

print(df_question_answer.iloc[2])

#['답변']의 텍스트를 ' , ' 로 구분 후 질문에 하나씩 쌓기
sent1_list, sent2_list, label_list, index_list = [], [], [], []
for i in range(len(df_question_answer)):
  row = df_question_answer.iloc[i]
  question_text = row['내용']
  answer_text_list = row['답변'].split(' , ')
  for j in range(len(answer_text_list)):
    if answer_text_list[j] != '':
      sent1_list.append(question_text + ''.join(answer_text_list[:j]))
      sent2_list.append(answer_text_list[j])
      label_list.append(1)
      index_list.append(i)

finetune_dict = {'sent1' : sent1_list, 'sent2' : sent2_list, 'label' : label_list, 'sent2_index' : index_list}

df_community_finetune = pd.DataFrame.from_dict(finetune_dict)
print(df_community_finetune)

Unnamed: 0                                                    4
내용            안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...
답변            충분히 버블로 개발가능할 것 같아요! 이미 비슷한 수준의 어플을 개발한 사례가 많아...
댓글            버블은 다 되요. 개인별 능력이 좌우합니다. 치킨은 살 안찌듯. , 1:1 매칭하는...
관련 링크                                                          
작성일                                         2023-01-12 19:01:00
작성자                                                         김정민
제목                                         버블로 소개팅 앱 만들 수 있을까요?
첨부 파일                                                          
Slug                                        버블로-소개팅-앱-만들-수-있을까요
게시글 링크                                                         
Name: 4, dtype: object
                                                 sent1  \
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3    안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있

In [ ]:
#파인튜닝 데이터셋에서 n배의 negative sample을 생성하는 코드
def create_negative_sample(DataFrame, n):
  DataFrame_copy = DataFrame.copy() #원본 데이터셋을 변화시키지 않기 위해 deep copy
  length = len(DataFrame_copy)
  for i in range(length):
    row = DataFrame_copy.iloc[i]
    negative_candidate_ids = list(set(DataFrame_copy['sent2_index'].tolist())) #.remove(row['sent2_index']) #해당 행의 sent2를 negative sample 후보군에서 제외
    negative_candidate_ids.remove(row['sent2_index'])
    negative_sample_ids = list(np.random.choice(negative_candidate_ids, n, replace = False))
    for idx in negative_sample_ids:
      new_row = pd.Series({'sent1' : row['sent1'], 'sent2' : np.random.choice(DataFrame_copy[DataFrame_copy['sent2_index'] == idx]['sent2'].tolist(), 1)[0], 'label' : 0, 'sent2_index' : idx}) #카톡 데이터는 idx 중복이 없기 때문에 랜덤추출로 1개를 뽑아도 달라지는 건 없고, 커뮤니티 데이터의 경우 idx 중복이 있기 때문에 해당 idx를 가진 sent2 중 하나를 뽑아야 함.
      DataFrame_copy = pd.concat([DataFrame_copy, new_row.to_frame().T], ignore_index = True)
  return DataFrame_copy

In [ ]:
#커뮤니티 파인튜닝 데이터셋에서 negative sample 생성
df_community_finetune_negative_sampled = create_negative_sample(df_community_finetune, 7)
# df_community_finetune_negative_sampled = df_community_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_community_finetune_negative_sampled))
print(df_community_finetune_negative_sampled)

1824
                                                  sent1  \
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
4     모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
...                                                 ...   
1819  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1820  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1821  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1822  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1823  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   

                                                  sent2 label sent2_index  
0     저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...     1           0  
1     헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...     1           1  
2     모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 ...   

In [ ]:
print(df_community_finetune_negative_sampled[df_community_finetune_negative_sampled['sent2'] == ''])
print(df_community_finetune[df_community_finetune['sent2_index'] == 13])

Empty DataFrame
Columns: [sent1, sent2, label, sent2_index]
Index: []
                                                sent1  \
15  플러스 버튼 (+) 을 클릭하면 리피팅 그룹 내의 로우가 +1 되도록 리피팅 그룹의...   
16  플러스 버튼 (+) 을 클릭하면 리피팅 그룹 내의 로우가 +1 되도록 리피팅 그룹의...   

                                                sent2  label  sent2_index  
15  일단 제가 소영님의 질문을 정확히 이해했는지 잘 모르겠습니다. 질문대로라면 리피팅그...      1           13  
16  저는 플러그인 요청하셔서 간단하게 제가 변형해서 쓰는 템플릿 살포시 투척 해놓고 가...      1           13  


In [ ]:
print(df_talk_finetune)

                                                 sent1  \
0     deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...   
1     유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...   
2     혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
3     혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
4     혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
..                                                 ...   
496  각 element에다가 condition을 따로 쓰면 workflow에 안들어가있기...   
497  각 element에다가 condition을 따로 쓰면 workflow에 안들어가있기...   
498   안녕하세요, 혹시 버블에서 three.js라는 라이브러리 연결을 도와주실 분 계실...   
499   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
500   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                 sent2  label  sent2_index  
0     익스프레션 내에서 조건식을 사용할 수가 없어서 condition을 사용하는 수밖에...      1            1  
1     이게 근데 한번 락인되니까 DB 이관한다는 선택지를 고르기가 참 어렵네요.. 또 ...      0           13  
2     1. Do a search for 할 때 constraints 걸어서 검색량 줄이...      1           

In [ ]:
#카카오톡 파인튜닝 데이터셋에서 negative sample 생성
df_talk_finetune_negative_sampled = create_negative_sample(df_talk_finetune, 7)
df_talk_finetune_negative_sampled = df_talk_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_talk_finetune_negative_sampled))
print(df_talk_finetune_negative_sampled)

4000
                                                  sent1  \
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...   
1      유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...   
2      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
3      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
4      혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
...                                                 ...   
3995   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3996   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3997   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3998   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3999   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                  sent2 label  
0      익스프레션 내에서 조건식을 사용할 수가 없어서 condition을 사용하는 수밖에...     1  
1      이게 근데 한번 락인되니까 DB 이관한다는 선택지를 고르기가 참 어렵네요.. 또 ...     0  
2      1. Do a search for 할 때 constraints 걸어서 검색량 줄이...     1  
3                    Do a sear

In [ ]:
print(df_talk_finetune_negative_sampled[df_talk_finetune_negative_sampled['sent2'] == ''])

Empty DataFrame
Columns: [sent1, sent2, label]
Index: []


In [ ]:
#두 파인튜닝 데이터셋을 하나로 합치기
df_KcBERT_finetune = pd.concat([df_community_finetune_negative_sampled, df_talk_finetune_negative_sampled], ignore_index = True)
print(len(df_KcBERT_finetune))

5824


In [ ]:
#.xlsx 파일로 저장
finetune_data_save_path = '/content/drive/My Drive/KcBERT_finetune_dataset_answer.xlsx'

df_KcBERT_finetune.to_excel(finetune_data_save_path)

#Finetune 데이터셋(질문)

왜 빌더로그 게시글 요소들을 세 부분으로 나눠야 하는가?
1. positive sample, negative sample 비율 1대1이 되도록 만들기
2. 사용할 RobertaForSequenceClassification 아키텍처를 보면 (position_embeddings): Embedding(514, 768, padding_idx=1)이 포함되어 있다. negative sample에 해당하는 빌더로그 게시글만 길이가 길다면 이러한 정보까지 학습할 여지가 있다.

왜 tokenizer()로 바로 시퀀스 길이를 구하지 않고 원본 리스트부터 시작해서 평균 길이를 구하는가
->

인자 중 max_length가 존재하기 때문에 어차피 길이를 초과하는 데이터는 truncation이 진행된다.

그렇기 때문에 데이터 평균 길이를 비교하려면 원본 리스트를 사용해야 한다.

In [ ]:
#토큰화를 위한 pretrained model 로드
pretrained_model_path = '/content/drive/My Drive/Pretrained_Model'

pretrained_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
pretrained_model = BertForSequenceClassification.from_pretrained(pretrained_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/Pretrained_Model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#제작한 데이터셋의 결측치 제거
df_talk_finetune_question = df_talk_finetune_question[(pd.notna(df_talk_finetune_question['text'])) & (pd.notna(df_talk_finetune_question['label']))]
print(df_talk_finetune_question)
df_talk_finetune_question_positive = df_talk_finetune_question[df_talk_finetune_question['label'] == 1]
print(df_talk_finetune_question_positive)

                                                   text  label
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...    1.0
1      익스프레션 내에서 조건식을 사용할 수가 없어서 condition을 사용하는 수밖에...    0.0
2      유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...    1.0
3      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...    1.0
4                      으헉 .. 백엔드만 사용하셔도 그렇게 많이 나오는군요 ..    0.0
...                                                 ...    ...
2996   답변 감사합니다~ 온보딩을 인덱스 페이지로 만드셨다는 말씀이시죠? 저도 고민해봐야...    0.0
2997                               중개서비스 준비중인 대표님두 계신가요    0.0
2998                    html 사이트 괜첞은곳 추천좀 부탁드립니다 고수님들!😁    1.0
2999        혹시 지금 버블 에디터 불안정한가요? field가 보였다 말았다 그러네요하;;    1.0
3000   모바일에서 Reapting group의 하나에 피드(셀)를 공유하고 공유받은사람이...    1.0

[2932 rows x 2 columns]
                                                   text  label
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...    1.0
2      유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...    1.0
3      혹시 버블 워크로드 최적화하는 팁이 있을까

In [ ]:
#데이터셋의 tokenized data의 길이를 구하는 함수
def tokenized_sequence_length(text):
  #kss 모듈로 글을 문장으로 분리
  original_sentences = split_sentences(text)
  index_sentences = pretrained_tokenizer(original_sentences)['input_ids']
  index_sentences_filtered = [sentence[1:-1] for sentence in index_sentences] #cls_token과 sep_token 제외
  total_length = sum(len(s) for s in index_sentences_filtered)
  return index_sentences_filtered, total_length

In [ ]:
#데이터셋에서 각 요소를 n등분하는 함수
def divide_text_to_number(text, number):
  #kss 모듈로 문장으로 분리
  index_sentences, total_length = tokenized_sequence_length(text)
  target_length = total_length / number

  # 세 부분으로 분할
  segments = []
  current_segment = []
  current_length = 0

  #마지막 세그먼트 추가를 위해 사용하는 인덱스 값
  sentence_checkpoint = 0

  for index, sentence in enumerate(index_sentences):
      if current_length + len(sentence) > target_length and current_segment:
          segments.append(sum(current_segment, []))
          current_segment = [sentence]
          current_length = len(sentence)
          if len(segments) == number - 1:  # 이미 (목표 세그먼트 개수 - 1)개 세그먼트를 만들었다면 나머지는 모두 마지막 세그먼트에 추가
              sentence_checkpoint = index #current_segment에 추가해둔 문장을 인덱싱
              break
      else:
          current_segment.append(sentence)
          current_length += len(sentence)

  # 마지막 세그먼트 추가
  segments.append(sum(current_segment + index_sentences[sentence_checkpoint + 1:], []))

  return pretrained_tokenizer.batch_decode(segments)


In [ ]:
#BP_judge_question용 데이터 제작

#slug가 있다면 질문, 없다면 답변, 이 중 질문을 positive sample로 설정
question_list = df_qna[pd.isna(df_qna['Slug']) == False]['내용'].dropna(axis = 0).to_list()
label_question = [1] * len(question_list)

print(question_list)
print('데이터셋의 전체 길이 :', len(question_list))

#질문 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(question) for question in question_list) / len(question_list))

#질문 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in question_list) / len(question_list))

['안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 쓰게 되네요!서베이 폼을 이용해 받은 설문 내용을 노션 데이터베이스로 가져온 후 자피어를 활용하여 데이터베이스의 각 속성 값을 새로운 페이지의 글로 작성하려 하는데 일반적인 딱딱한 텍스트만 있는 글이 아니라 기본적으로 노션에서 사용할 수 있는 콜아웃 블록이나 인용 등의 기능을 구현하고 싶은데 자피어에서 새로운 페이지를 작성하는 필드에 노션 기능을 구현할 수 있는 명령어나 언어가 따로 있는지 궁금해서 이렇게 질문을 남깁니다.이 고민을 몇시간째 하고 있는데 사이트 오픈 소식 메일이 와서 사이트를 둘러 보다가 이렇게 글 남깁니다.다시 한번 오픈 축하드립니다.감사합니다.', '안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 페이지를 웹뷰로보면 양측면으로 여백이 있는데요, 제가 현재 개발하고 있는 페이지에는 여백이 적용되지않습니다.각 페이지마다 사이드그룹을 지정해서 여백을 두신건지.. 다른 세팅 방법이 있는지 궁금합니다.현재 제가 작업중인 페이지를 이미지로 첨부합니다. 헤더가 왼쪽에서 오른쪽까지 가득 차있어요. (혹시 테스트뷰만 이렇게 나오는건 아닐까라는 기대도 해보게되네요..)', '안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나 이후로 자극 받아 버블로 다시 시작하려 합니다!사실 노션에서 기능이 너무 한정적이라 답답함이 있었고, 특정 기능은 굉장히 높은 코딩 스킬을 요구한다는 것을 알게 되었습니다ㅜMVP앱 기능을 아래처럼 구상중입니다.사용자 자기소개서(서베이 폼 Tally) 작성사용자 응답에서 각 요소별 답변을 데이터베이스에 저장데이터베이스를 제가공(자동화?)하여 프로필 페이지 생성프로필 내용 일부 공개, 전체 열람 유료 결제1:1 대화신청 (카카오톡 1:1 대화방 or 자체 채팅기능?) 유료 결제답변 요소 중 특정 요소만 따로 모아 놓은 페이지 생성 (예, 유튜브 추천 영상 모음 페이지)매주 1회 답변 요소 중

In [ ]:
#답변을 negative sample로 설정
answer_list = df_qna['답변'].dropna(axis = 0).to_list()
label_answer = [0] * len(answer_list)

print(answer_list)
print(len(answer_list))

#답변 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(answer) for answer in answer_list) / len(answer_list))

#답변 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in answer_list) / len(answer_list))

#시퀀스 길이 균형을 위해 각 답변 글을 n등분
answer_list_divided = sum([divide_text_to_number(text, 2) for text in answer_list], [])

#분할한 데이터의 개수에 맞춰서 레이블 설정
label_answer = [0] * (len(answer_list_divided))

#분리된 답변 글 평균 길이 확인(토큰화 결과)
print('분리된 데이터셋에서 토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in answer_list_divided) / len(answer_list_divided))

#분리된 데이터셋의 전체 길이
print('분리된 데이터셋의 전체 길이 :', len(answer_list_divided))

['저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못했고,마크다운을 지원하기 때문에 토글이나 체크리스트, 인용구 같은 부분까지는 꾸밀 수 있는 것 같습니다.이쪽 블로그에 자세히 나와있는 것 같아요.콘텐츠 부분에 마크다운을 미리 채워두시고 내용물은 동적 데이터로 지정하시면 깔끔하게 떨어지지 않을까 싶네요. (혹시 이미 해보셨으려나요?;;)저도 해보진 않은 거라서 어디까지나 추측이고 가능성입니다. 정민님이 해보시고 도움 되셨다면 빌더 로그 같은 곳에 공유 부탁드립니다!', '헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요! 그리고 헤더그룹의 Horizontal Alignment를 centered로 두시면 그룹이 페이지 중앙에 위치하고 설정해두신 사이즈까지만 늘어나면서 여백이 생길 것 같아요!화면 해상도 관련해서는 아래에서 참고하시면 될 것 같습니다 , 모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 경우 페이지 width를 1080px로 하고 바디 그룹의 레이아웃을 row로 했고, 다시 그 안에 column 그룹을 2개 추가하여 레이아웃을 다음 이미지와 같이 짰습니다~', '충분히 버블로 개발가능할 것 같아요! 이미 비슷한 수준의 어플을 개발한 사례가 많아서 참고하실 레퍼런스도 많을 것 같아요.혹시 시간이 걸리실 것 같으면 매칭, 채팅 같은 핵심기능만 먼저 개발하셔서 테스트 돌려보시면서 지표 보시는 것도 좋을 것 같아요! 마침 tinder를 클론할 수 있도록 버블이 가이드를 제공한 페이지가 있어 공유드립니다.혹시 MVP 출시하시면 꼭 알려주세요 ㅎㅎ 제주도 사는 친구들에게 공유해보고 싶네요 :)', '폴리미 가입해야지 가입해야지 하다가 바로 가입했습니다. 정신이 없었네요. 좋은 프로젝트라서 기대가 됩니다. 따지고보면 저도 모두의노코드 모더레이터가 부캐거든요!1) 모두의노코드는 페이지별로 모바일용 페이지를 따로 만드셨나요?A : 중요한 페이지만 모바일용으로 따로 

In [ ]:
#빌더로그 글을 negative sample로 설정
builder_log_list = df_all_contents[df_all_contents['커뮤니티 타입'] == '빌더 로그']['내용'].dropna(axis = 0).to_list()

print(builder_log_list)
print(len(builder_log_list))

#빌더로그 글 평균 길이 확인(원본 텍스트)
print('원본 텍스트 평균 길이 :', sum(len(builder_log) for builder_log in builder_log_list) / len(builder_log_list))

#빌더로그 글 평균 길이 확인(토큰화 결과)
print('토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list) / len(builder_log_list))

#시퀀스 길이 균형을 위해 각 빌더로그 글을 n등분
builder_log_list_divided = sum([divide_text_to_number(text, 6) for text in builder_log_list], [])

#분할한 데이터의 개수에 맞춰서 레이블 설정
label_builder_log = [0] * (len(builder_log_list_divided))

#분리된 빌더로그 글 평균 길이 확인(토큰화 결과)
print('분리된 데이터셋에서 토큰화된 텍스트 평균 길이 :', sum(tokenized_sequence_length(text)[1] for text in builder_log_list_divided) / len(builder_log_list_divided))

#분리된 데이터셋의 전체 길이
print('분리된 데이터셋의 전체 길이 :', len(builder_log_list_divided))

["어제 프론티어 분들께만 모두의노코드 오픈 소식을 하루 먼저 알려드리고 불과 몇 시간 만에 서버 용량이 거의 찼다는 메일을 받았습니다.웨비나 중이어서 메일을 미처 확인하지 못했는데요. 약 30여 명 정도 가입한 시점에 서버 용량이 다 차버릴 줄은 꿈에도 몰랐습니다. 퍼스널 요금제가 정말 그야말로 '퍼스널'이었네요.그래서 5분 정도가 가입하실 때 워크플로우가 제대로 작동하지 않아 가입은 되셨는데 데이터가 제대로 들어가지 않았습니다. 심심한 사과의 말씀드립니다. 이분들은 따로 연락을 드려 조치를 취할 예정입니다.정리 : 커뮤니티 제작하실 때 되도록 퍼스널 요금제는 피하시는 게 좋을 것 같습니다.", '슬랙 알림 기능은 비교적 간단하며, 무료입니다.api connector을 이용하며, 이것을 이용해 워크플로우 상에서 특정 채널에 내가 원하는 메세지를 보낼 수 있습니다.먼저 Plugin에서 API Connector을 설치하세요위와 같은 형태로 api를 추가하면 됩니다. 빨간색 박스로 표시한 설정을 아래와 같이 따라해보세요.Use as ‘Action’Data type ‘Text’POST (드롭다운)Body type : JSONBody : {”text”: “<text>” }Body parameters - value : 아무 텍스트 (테스트용이라서 아무렇게나 입력하셔도 됩니다), Private 체크 해제이제 초록색 부분에 api url을 입력하면 되는데, 핑크색 가려진 부분은 내가 슬랙 메세지를 보내고 싶은 채널의 고유 id를 가져와야 합니다. 방법은 아래에서 계속.위 링크로 접속하면, 수신 웹후크 앱이 나옵니다. 이제 이 앱을 ‘Slack에 추가’ 합니다.로그인 한 상태에서, 포스트할 채널을 드롭다운에서 선택합니다.이제 웹후크 URL이라는 것이 나옵니다. 이것을 앞에서 설명드린 초록색 박스에 복사붙여넣기 하면 됩니다.Initialize를 누르면, ok 라는 텍스트가 나옵니다. (정상적으로 완료된 상태)이제 워크플로우 상에서 등록한 api를 사용해 슬랙에 메세지를 보낼 

In [ ]:
#커뮤니티 글 데이터프레임 생성
text = question_list + answer_list + builder_log_list_divided
label = label_question + label_answer + label_builder_log

df_judge_question = pd.DataFrame([text, label], index = ['text', 'label'])
df_judge_question = df_judge_question.T
df_judge_question = df_judge_question[pd.isna(df_judge_question['text']) == False]
print(df_judge_question)


                                                  text label
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...     1
1    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...     1
2    안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...     1
3                                 API키는 분리해서 쓰시면 좋겠습니다     1
4                                                    .     1
..                                                 ...   ...
809  특히 나 영어 권 개발 사와 이메일과 줌 으로 만 소 통해 야 하는 것은 프로젝트 ...     0
810  스티비 는 국내 뉴스레터 서비스로, 카페24, 고도몰, 식스샵 등 다양한 서비스와 ...     0
811   하지만 버블에서 딱히 연동할 수 있는 게 없어서 무료 플러그인을 작업하고 있습니다. _     0
812                          x0008_ 이번 주에 무료 배포할 예정입니다     0
813  :) + 필요한 플러그인이 있다면, 버블박스 사이트에서 알려주세요! 하지만 버블에서...     0

[814 rows x 2 columns]


In [ ]:
#직접 제작한 카카오톡 데이터프레임과 결합
df_KcBERT_finetune_question = pd.concat([df_judge_question, df_talk_finetune_question], ignore_index = True)
print(df_KcBERT_finetune_question)

                                                   text label
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...     1
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...     1
2     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...     1
3                                  API키는 분리해서 쓰시면 좋겠습니다     1
4                                                     .     1
...                                                 ...   ...
3584   안녕하세요. 왕초보자입니다. 리피팅 그룹을 따라 만들어보고 있는데 저는 스크롤에 ...   1.0
3585                                 영상의 에디터가 옛날버전인것같네요   0.0
3586   그래서 저는 이것저것에서 스크롤 기능을찾아보는데 없네요... 업데이트 되면서 리피...   1.0
3587   지금은 1*4로. 리피팅 그룹이 고정된 상태시네요 가로 스크롤을 구현하려고 하시는...   0.0
3588             그럼 row값을 1로 고정하시고 column은 Fixed를 푸셔야해요   0.0

[3589 rows x 2 columns]


In [ ]:
#.xlsx 파일로 저장
finetune_data_save_path = '/content/drive/My Drive/KcBERT_finetune_dataset_question.xlsx'

df_KcBERT_finetune_question.to_excel(finetune_data_save_path)